In [1]:
import pandas as pd
df = pd.read_csv('file-for-fineTuning.csv')
df.head()

,text,label,ticket_id,agent_name,text_length
0,Where can I find API rate limits documentation?,other,1000,Dana,47
1,Can I migrate data from Freshdesk?,other,1001,Bob,34
2,Cannot update billing address; page keeps relo...,billing,1002,Charlie,52
3,Looking for a product roadmap or upcoming feat...,other,1003,Dana,51
4,Dark mode toggled but UI stays light.,tech_support,1004,Alice,37


In [2]:
texts = df["text"].tolist()
labels = df["label"].astype("category").cat.codes.tolist()

Tokenize the Text Data

In [3]:
### Using the DistilBERT tokenizer:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
encodings = tokenizer(texts, truncation = True, padding = True, max_length = 128)

c:\Users\HUMAIDU\Desktop\groub B\Thrive_Internship_ML_A\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
c:\Users\HUMAIDU\Desktop\groub B\Thrive_Internship_ML_A\venv\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Creating Dataset for Training

In [4]:
from torch.utils.data import Dataset

class TicketDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labes'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

import torch
dataset = TicketDataset(encodings, labels)

In [5]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labes = train_test_split(texts, labels, test_size=0.2, random_state=42)
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

train_dataset = TicketDataset(train_encodings, train_labels)
val_dataset = TicketDataset(val_encodings, val_labes)